In [ ]:
import csv
import time
import os
from openai import OpenAI
from datetime import datetime

# API Configuration
API_PROVIDER = "openai"  # Options: "openai" or "openrouter"

# API Keys (replace with your actual keys)
OPENAI_API_KEY = "your_openai_key_here"
OPENROUTER_API_KEY = "your_openrouter_key_here"

# Setup OpenAI client based on selected provider
if API_PROVIDER == "openai":
    client = OpenAI(api_key=OPENAI_API_KEY)
    MODEL_NAME = "gpt-4.1"  # Use available OpenAI model
elif API_PROVIDER == "openrouter":
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=OPENROUTER_API_KEY
    )
    MODEL_NAME = "anthropic/claude-3-7-sonnet"  # OpenRouter model name
else:
    raise ValueError(f"Invalid API_PROVIDER: {API_PROVIDER}")

# Execution parameters
REPETITION = 3  # Number of times to repeat the prompt
PROMPT_TYPE = 1  # Select which prompt to use (1-4)

# Define the prompts
prompts = {
    1: """
Task: Generate pairwise comparison matrices and criteria weights for solar power plant site selection using the Analytic Hierarchy Process (AHP).

Context: Solar power plant site selection is a complex spatial decision problem requiring consideration of multiple criteria across technical, environmental, economic, and social dimensions. Your task is to perform pairwise comparisons of 14 criteria to determine their relative importance for optimal solar power plant siting in Şanlıurfa Province, Turkey, a region with high solar potential but diverse landscape features.

Instructions: You will use Saaty's AHP 1-9 scale for pairwise comparisons where:
1 = Equal importance
3 = Moderate importance of one over another
5 = Strong importance
7 = Very strong importance
9 = Extreme importance
2, 4, 6, 8 = Intermediate values
Reciprocals (e.g., 1/3, 1/5) indicate the inverse relationship

Criteria List:
1. Distance to Power Transmission Lines (m) 
2. Slope (%) 
3. Distance to Rivers and Lakes (m) 
4. Distance from Fault Lines (m) 
5. Distance to Protected Areas (m) 
6. Land Use 
7. Distance to Bird Migration Routes (m) 
8. Distance to Transformers (m) 
9. Aspect (Slope Direction) 
10. Altitude (m) 
11. Distance to Settlements (m) 
12. Solar Radiation (kWh/m²-year) 
13. Distance to Airports (m) 
14. Distance to Highways (m)

Output Format: Please Provide:
1. A complete 14×14 pairwise comparison matrix with all pairwise judgments (aij) where i = row criterion and j = column criterion, formatted as a NumPy array that can be directly copied into Python code.
2. Calculated priority weights for each criterion (normalized eigenvector)
3. Consistency Ratio (CR) calculation to verify judgment consistency (should be <0.1)

Approach the task as if you were synthesizing the judgments of multiple experts in renewable energy planning, considering the technical feasibility, economic viability, environmental sustainability, and social acceptability of potential solar power plant locations.
""",
    
    2: """
Task: Generate Analytic Hierarchy Process (AHP) criteria weights for solar power plant site selection through systematic pairwise comparisons.

Context: Solar power plant siting requires balancing technical, economic, environmental, and social factors to identify optimal locations. This analysis focuses on Şanlıurfa Province, Turkey, a region with significant solar potential and diverse landscape characteristics.

Instructions: Apply Saaty's AHP methodology to determine relative importance of 14 siting criteria. For each pairwise comparison:
1. Explicitly state your reasoning process
2. Consider the technical purpose of each criterion
3. Evaluate the relative contribution to overall site suitability
4. Assign a numerical value using Saaty's 1-9 scale:
   - 1: Equal importance
   - 3: Moderate importance
   - 5: Strong importance
   - 7: Very strong importance
   - 9: Extreme importance
   - 2,4,6,8: Intermediate values
   - Reciprocals (1/3, 1/5, etc.): Inverse relationships

Criteria Details:
1. Distance to Power Transmission Lines (m): Proximity to grid connection, with distances <5000m economically advantageous.
2. Slope (%): Gradient of terrain, with gentle slopes (0-3%) optimal for construction and maintenance.
3. Distance to Rivers and Lakes (m): Protection of water resources, with distances >2000m considered environmentally appropriate.
4. Distance from Fault Lines (m): Safety buffer from geological hazards, with distances >2000m considered highly suitable.
5. Distance to Protected Areas (m): Separation from environmentally sensitive zones, with distances >2000m considered suitable.
6. Land Use: Compatibility of existing land use with solar deployment, with barren rocky areas most suitable, while protected, residential, and agricultural lands are excluded.
7. Distance to Bird Migration Routes (m): Protection buffer for avian wildlife, with distances >5000m considered highly suitable.
8. Distance to Transformers (m): Accessibility to electrical infrastructure, with distances <5000m considered optimal.
9. Aspect (Slope Direction): Orientation of terrain, with south-facing and flat areas most suitable for northern hemisphere installations.
10. Altitude (m): Elevation above sea level, with higher elevations (1410-1955m) preferred due to reduced atmospheric interference.
11. Distance to Settlements (m): Buffer from rural habitation areas, with distances >1500m preferred to minimize social impacts.
12. Solar Radiation (kWh/m²-year): Primary energy resource potential, with values above 1800 considered highly suitable.
13. Distance to Airports (m): Safety buffer from aviation operations, with distances >5000m preferred to avoid interference.
14. Distance to Highways (m): Accessibility for construction and maintenance, with 100-2500m range considered highly suitable.

Step-by-Step Process:
1. First, reason through the general importance hierarchy among the four main criteria categories (resource potential, topographical suitability, environmental constraints, infrastructure connectivity)
2. For each pairwise comparison, explicitly articulate:
   - The function/purpose of each criterion in solar plant siting
   - Their relative contribution to technical feasibility
   - Their relative contribution to economic viability
   - Their relative contribution to environmental sustainability
   - Their relative contribution to social acceptability
   - The final judgment value based on this multifaceted analysis
3. After completing the pairwise comparison matrix, calculate the priority weights using the eigenvector method
4. Verify the consistency of your judgments by calculating the Consistency Ratio

Example Reasoning (for illustration):
"When comparing Solar Radiation to Land Use, I consider that solar radiation directly determines energy production potential, the primary function of the facility, while land use represents a constraint rather than a production factor. Solar radiation has a fundamental impact on economic viability through energy yield, whereas appropriate land can be acquired through various means. Therefore, I judge Solar Radiation to be strongly more important than Land Use (value: 5)."

Output Format: Please provide:
1. A complete 14×14 pairwise comparison matrix with all pairwise judgments (aij) where i = row criterion and j = column criterion, formatted as a NumPy array that can be directly copied into Python code.
2. Calculated priority weights for each criterion (normalized eigenvector)
3. Consistency Ratio (CR) calculation to verify judgment consistency (should be <0.1)

Approach the task as if you were synthesizing the judgments of multiple experts in renewable energy planning, considering the technical feasibility, economic viability, environmental sustainability, and social acceptability of potential solar power plant locations.
""",
    
    3: """
Task: As a diverse panel of renewable energy experts, generate AHP criteria weights for optimal solar power plant site selection.

Expert Panel Role Context: You represent a multidisciplinary committee of twelve specialists in renewable energy planning, including:
- Solar energy engineers with technical expertise in photovoltaic systems
- Environmental scientists specializing in ecological impact assessment
- Spatial planners with experience in renewable energy infrastructure
- Energy economists focused on renewable project feasibility
- Local government representatives from Şanlıurfa Province, Turkey

Your collective task is to determine the relative importance of 14 criteria for solar power plant siting through structured pairwise comparisons using the Analytic Hierarchy Process.

Problem Context: Şanlıurfa Province in southeastern Turkey has been identified as a high-potential region for utility-scale solar development due to its excellent solar radiation (exceeding 1600 kWh/m²-year in many areas), suitable topography, and growing energy demand. Your committee must balance technical performance, economic feasibility, environmental protection, and social considerations to identify optimal sites for development.

Instructions: Using your collective expertise, perform pairwise comparisons of all criteria using Saaty's 1-9 scale:
1 = Equal importance
3 = Moderate importance of one over another
5 = Strong importance
7 = Very strong importance
9 = Extreme importance
2, 4, 6, 8 = Intermediate values
Reciprocals (e.g., 1/3, 1/5) indicate the inverse relationship

For each comparison, consider:
- The expert consensus across disciplines
- Regional characteristics specific to Şanlıurfa Province
- International best practices in solar siting
- Long-term sustainability considerations

Criteria Specifications:
1. Distance to Power Transmission Lines (m): Proximity to grid connection, with distances <5000m economically advantageous.
2. Slope (%): Gradient of terrain, with gentle slopes (0-3%) optimal for construction and maintenance.
3. Distance to Rivers and Lakes (m): Protection of water resources, with distances >2000m considered environmentally appropriate.
4. Distance from Fault Lines (m): Safety buffer from geological hazards, with distances >2000m considered highly suitable.
5. Distance to Protected Areas (m): Separation from environmentally sensitive zones, with distances >2000m considered suitable.
6. Land Use: Compatibility of existing land use with solar deployment, with barren rocky areas most suitable, while protected, residential, and agricultural lands are excluded.
7. Distance to Bird Migration Routes (m): Protection buffer for avian wildlife, with distances >5000m considered highly suitable.
8. Distance to Transformers (m): Accessibility to electrical infrastructure, with distances <5000m considered optimal.
9. Aspect (Slope Direction): Orientation of terrain, with south-facing and flat areas most suitable for northern hemisphere installations.
10. Altitude (m): Elevation above sea level, with higher elevations (1410-1955m) preferred due to reduced atmospheric interference.
11. Distance to Settlements (m): Buffer from rural habitation areas, with distances >1500m preferred to minimize social impacts.
12. Solar Radiation (kWh/m²-year): Primary energy resource potential, with values above 1800 considered highly suitable.
13. Distance to Airports (m): Safety buffer from aviation operations, with distances >5000m preferred to avoid interference.
14. Distance to Highways (m): Accessibility for construction and maintenance, with 100-2500m range considered highly suitable.

Output Format: Please provide:
1. A complete 14×14 pairwise comparison matrix with all pairwise judgments (aij) where i = row criterion and j = column criterion, formatted as a NumPy array that can be directly copied into Python code.
2. Calculated priority weights for each criterion (normalized eigenvector)
3. Consistency Ratio (CR) calculation to verify judgment consistency (should be <0.1)

Your task is to produce weights that reflect a balanced, expert-driven assessment of criteria importance for this specific geographic and developmental context.
""",
    
    4: """
Task: Generate pairwise comparison matrices and criteria weights for solar power plant site selection using the Analytic Hierarchy Process (AHP).

Context: Solar power plant site selection is a complex spatial decision problem requiring consideration of multiple criteria across technical, environmental, economic, and social dimensions. Your task is to perform pairwise comparisons of 14 criteria to determine their relative importance for optimal solar power plant siting in Şanlıurfa Province, Turkey, a region with high solar potential but diverse landscape features.

Instructions: You will use Saaty's AHP 1-9 scale for pairwise comparisons where:
1 = Equal importance
3 = Moderate importance of one over another
5 = Strong importance
7 = Very strong importance
9 = Extreme importance
2, 4, 6, 8 = Intermediate values
Reciprocals (e.g., 1/3, 1/5) indicate the inverse relationship

Criteria List:
1. Solar Radiation (kWh/m²-year)
2. Aspect (Slope Direction)
3. Land Use
4. Slope (%)
5. Distance to Power Transmission Lines (m)
6. Distance to Settlements (m)
7. Distance to Transformers (m)
8. Altitude (m)
9. Distance to Protected Areas (m)
10. Distance to Highways (m)
11. Distance from Fault Lines (m)
12. Distance to Airports (m)
13. Distance to Rivers and Lakes (m)
14. Distance to Bird Migration Routes (m)


Output Format: Please provide:
1. A complete 14×14 pairwise comparison matrix with all pairwise judgments (aij) where i = row criterion and j = column criterion, formatted as a NumPy array that can be directly copied into Python code.
2. Calculated priority weights for each criterion (normalized eigenvector)
3. Consistency Ratio (CR) calculation to verify judgment consistency (should be <0.1)

Approach the task as if you were synthesizing the judgments of multiple experts in renewable energy planning, considering the technical feasibility, economic viability, environmental sustainability, and social acceptability of potential solar power plant locations.
"""
}

prompt_names = {
    1: "Minimal_Context",
    2: "Chain-of-Thought", 
    3: "Expert_Panel",
    4: "Expert_Aligned"
}

def run_simple_ahp_collection():
    # Get the selected prompt
    prompt = prompts.get(PROMPT_TYPE, prompts[1])
    prompt_name = prompt_names.get(PROMPT_TYPE, "Unknown")
    
    # Create results directory
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_dir = f"{API_PROVIDER}_{MODEL_NAME.replace('/', '_')}_{prompt_name}_{timestamp}"
    os.makedirs(results_dir, exist_ok=True)

    # CSV file for responses
    csv_path = os.path.join(results_dir, "responses.csv")
    
    with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Run', 'Timestamp', 'Response'])
        
        for i in range(REPETITION):
            try:
                print(f"Making API call {i+1}/{REPETITION}...")
                
                # Make API call
                completion = client.chat.completions.create(
                    model=MODEL_NAME,
                    messages=[{"role": "user", "content": prompt}]
                )
                
                response_content = completion.choices[0].message.content
                current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                
                # Save to CSV
                writer.writerow([i+1, current_time, response_content])
                
                # Save individual response file
                response_file = os.path.join(results_dir, f"response_{i+1}.txt")
                with open(response_file, 'w', encoding='utf-8') as f:
                    f.write(f"Run: {i+1}\n")
                    f.write(f"Timestamp: {current_time}\n")
                    f.write(f"Model: {MODEL_NAME}\n")
                    f.write(f"Prompt Type: {prompt_name}\n")
                    f.write("-" * 50 + "\n\n")
                    f.write(response_content)
                
                print(f"✓ Run {i+1} completed successfully")
                
                # Brief pause between requests
                if i < REPETITION - 1:  # Don't sleep after the last request
                    time.sleep(1)
                
            except Exception as e:
                print(f"✗ Error on run {i+1}: {str(e)}")
                current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                writer.writerow([i+1, current_time, f"ERROR: {str(e)}"])
                time.sleep(2)  # Longer pause on error

    print(f"\n✓ All requests completed!")
    print(f"Results saved to: {results_dir}")
    print(f"- CSV file: {csv_path}")
    print(f"- Individual responses: response_1.txt, response_2.txt, etc.")

if __name__ == "__main__":
    run_simple_ahp_collection()